In [1]:
import ctypes as ctypes
import ctypes.wintypes
import sys
import time

import ctypes_wrappers

In [2]:
list_mouse = []

In [3]:
HWND_MESSAGE = -3

WM_QUIT = 0x0012
WM_INPUT = 0x00FF
WM_KEYUP = 0x0101
WM_CHAR = 0x0102

HID_USAGE_PAGE_GENERIC = 0x01

RIDEV_NOLEGACY = 0x00000030
RIDEV_INPUTSINK = 0x00000100
RIDEV_CAPTUREMOUSE = 0x00000200

RID_HEADER = 0x10000005
RID_INPUT = 0x10000003

RIM_TYPEMOUSE = 0
RIM_TYPEKEYBOARD = 1
RIM_TYPEHID = 2

PM_NOREMOVE = 0x0000

In [4]:
kernel32 = ctypes.WinDLL("Kernel32")
user32 = ctypes.WinDLL("User32")

In [5]:
HCURSOR = ctypes.c_void_p
LRESULT = ctypes.c_ssize_t

wndproc_args = (
    ctypes.wintypes.HWND,
    ctypes.wintypes.UINT,
    ctypes.wintypes.WPARAM,
    ctypes.wintypes.LPARAM,
)

WNDPROC = ctypes.CFUNCTYPE(LRESULT, *wndproc_args)

In [6]:
def structure_to_string_method(self):
    ret = [
        f"{self.__class__.__name__} (size: {ctypes.sizeof(self.__class__)}) instance at 0x{id(self):016X}:"
    ]
    for fn, _ in self._fields_:
        ret.append(f"  {fn}: {getattr(self, fn)}")
    return "\n".join(ret) + "\n"

class Struct(ctypes.Structure):
    to_string = structure_to_string_method

class WNDCLASSEXW(Struct):
    _fields_ = (
        ("cbSize", ctypes.wintypes.UINT),
        ("style", ctypes.wintypes.UINT),
        # ("lpfnWndProc", ctypes.c_void_p),
        ("lpfnWndProc", WNDPROC),
        ("cbClsExtra", ctypes.c_int),
        ("cbWndExtra", ctypes.c_int),
        ("hInstance", ctypes.wintypes.HINSTANCE),
        ("hIcon", ctypes.wintypes.HICON),
        ("hCursor", HCURSOR),
        ("hbrBackground", ctypes.wintypes.HBRUSH),
        ("lpszMenuName", ctypes.wintypes.LPCWSTR),
        ("lpszClassName", ctypes.wintypes.LPCWSTR),
        ("hIconSm", ctypes.wintypes.HICON),
    )

In [7]:
RegisterClassEx = user32.RegisterClassExW
RegisterClassEx.argtypes = (ctypes.POINTER(WNDCLASSEXW),)
RegisterClassEx.restype = ctypes.wintypes.ATOM

In [8]:
def wnd_proc(hwnd, msg, wparam, lparam):
    #print(f"Handle message - hwnd: 0x{hwnd:016X} msg: 0x{msg:08X} wp: 0x{wparam:016X} lp: 0x{lparam:016X}")
    if msg == WM_INPUT:
        size = ctypes.wintypes.UINT(0)
        res = ctypes_wrappers.GetRawInputData(
            ctypes.cast(lparam, ctypes_wrappers.PRAWINPUT),
            RID_INPUT,
            None,
            ctypes.byref(size),
            ctypes.sizeof(ctypes_wrappers.RAWINPUTHEADER),
        )

        buf = ctypes.create_string_buffer(size.value)
        res = ctypes_wrappers.GetRawInputData(
            ctypes.cast(lparam, ctypes_wrappers.PRAWINPUT),
            RID_INPUT,
            buf,
            ctypes.byref(size),
            ctypes.sizeof(ctypes_wrappers.RAWINPUTHEADER),
        )

        ri = ctypes.cast(buf, ctypes_wrappers.PRAWINPUT).contents
        head = ri.header
        
        if head.dwType == RIM_TYPEMOUSE:
            data = ri.data.mouse
            global list_mouse
            # list_mouse.append([data.mouse.LlastX, data.mouse.LlastY, data.ulRawButtons, data.ulButtons, data.usFlags])
            #print(data.to_string())
            list_mouse.append((data.lLastX, data.lLastY))
        elif head.dwType == RIM_TYPEKEYBOARD:
            data = ri.data.keyboard
            if data.VKey == 0x1B:
                ctypes_wrappers.PostQuitMessage(0)
        elif head.dwType == RIM_TYPEHID:
            data = ri.data.hid
        #print(data.to_string())
    return ctypes_wrappers.DefWindowProc(hwnd, msg, wparam, lparam)

In [9]:
def register_devices(hwnd=None):
    flags = RIDEV_INPUTSINK  # @TODO - cfati: If setting to 0, GetMessage hangs
    generic_usage_ids = (0x01, 0x02, 0x04, 0x05, 0x06, 0x07, 0x08)
    devices = (ctypes_wrappers.RawInputDevice * len(generic_usage_ids))(
        *(
            ctypes_wrappers.RawInputDevice(HID_USAGE_PAGE_GENERIC, uid, flags, hwnd)
            for uid in generic_usage_ids
        )
    )
    # for d in devices: print(d.usUsagePage, d.usUsage, d.dwFlags, d.hwndTarget)
    ctypes_wrappers.RegisterRawInputDevices(devices, len(generic_usage_ids), ctypes.sizeof(ctypes_wrappers.RawInputDevice))

In [10]:
wnd_cls = "SO049572093_RawInputWndClass"
wcx = ctypes_wrappers.WNDCLASSEX()
wcx.cbSize = ctypes.sizeof(ctypes_wrappers.WNDCLASSEX)
# wcx.lpfnWndProc = ctypes.cast(ctypes_wrappers.DefWindowProc, ctypes.c_void_p)
wcx.lpfnWndProc = ctypes_wrappers.WNDPROC(wnd_proc)
wcx.hInstance = ctypes_wrappers.GetModuleHandle(None)
wcx.lpszClassName = wnd_cls

In [11]:
res = ctypes_wrappers.RegisterClassEx(ctypes.byref(wcx)) # ctypes.byref() -> 주어진 객체에 대한 포인터 생성 : 매개변수로 포인터 전달해야 할 때 유용

'''
windows 시스템에서 새로운 윈도우 클래스를 등록

윈도우 클래스는 윈도우의 속성과 동작을 정의합니다. RegisterClassExW 함수를 사용하여 등록된 윈도우 클래스는 CreateWindowEx 함수를 호출하여 윈도우를 생성할 때 사용됩니다.

인자 -> WNDCLASSEX 구조체: 윈도우 클래스의 정보를 담고 있습니다. 이 정보에는 윈도우 프로시저(윈도우 이벤트를 처리하는 함수)와 윈도우 클래스의 이름 등이 포함됩니다.

반환 -> 성공하면 새로 등록된 윈도우 클래스의 고유 식별자(ATOM)를 반환하며, 실패하면 0을 반환합니다.
'''


'\nwindows 시스템에서 새로운 윈도우 클래스를 등록\n\n윈도우 클래스는 윈도우의 속성과 동작을 정의합니다. RegisterClassExW 함수를 사용하여 등록된 윈도우 클래스는 CreateWindowEx 함수를 호출하여 윈도우를 생성할 때 사용됩니다.\n\n인자 -> WNDCLASSEX 구조체: 윈도우 클래스의 정보를 담고 있습니다. 이 정보에는 윈도우 프로시저(윈도우 이벤트를 처리하는 함수)와 윈도우 클래스의 이름 등이 포함됩니다.\n\n반환 -> 성공하면 새로 등록된 윈도우 클래스의 고유 식별자(ATOM)를 반환하며, 실패하면 0을 반환합니다.\n'

In [12]:
hwnd = ctypes_wrappers.CreateWindowEx(
    0, wnd_cls, None, 0, 0, 0, 0, 0, 0, None, wcx.hInstance, None
)
register_devices(hwnd)

In [13]:
GetMessage = user32.GetMessageW
GetMessage.argtypes = (
    ctypes.wintypes.LPMSG,
    ctypes.wintypes.HWND,
    ctypes.wintypes.UINT,
    ctypes.wintypes.UINT,
)
GetMessage.restype = ctypes.wintypes.BOOL


TranslateMessage = user32.TranslateMessage
TranslateMessage.argtypes = (ctypes.wintypes.LPMSG,)
TranslateMessage.restype = ctypes.wintypes.BOOL

DispatchMessage = user32.DispatchMessageW
DispatchMessage.argtypes = (ctypes.wintypes.LPMSG,)
DispatchMessage.restype = LRESULT

In [14]:
msg = ctypes.wintypes.MSG()
pmsg = ctypes.byref(msg)

In [27]:
GetMessage(pmsg, None, 0, 0)
TranslateMessage(pmsg)
DispatchMessage(pmsg)

(1, 0, 0)


0

In [16]:
print("Start loop (press <ESC> to exit)...")
while res := GetMessage(pmsg, None, 0, 0):
    TranslateMessage(pmsg)
    DispatchMessage(pmsg)

Start loop (press <ESC> to exit)...
(0, 3, 0)
(7, 56, 0)
(7, 33, 0)
(9, 32, 0)
(9, 36, 0)
(10, 33, 0)
(13, 37, 0)
(10, 37, 0)
(9, 37, 0)
(6, 29, 0)
(6, 29, 0)
(6, 33, 0)
(5, 26, 0)
(4, 25, 0)
(2, 21, 0)
(2, 19, 0)
(1, 16, 0)
(2, 10, 0)
(1, 13, 0)
(1, 2, 0)
(0, 3, 7865344)
(1, 3, 0)
(6, 45, 0)
(0, 3, 0)
(1, 3, 7865344)
(0, 2, 0)
(6, 37, 0)
(1, 2, 7865344)
(3, 17, 0)
(4, 12, 0)
(0, 2, 7865344)
(0, 2, 0)
(11, 25, 0)
(3, 5, 0)
(1, 1, 7865344)
(1, 2, 0)
(5, 8, 0)
(4, 1, 0)
(5, 4, 0)
(2, 1, 0)
(1, 0, 7865344)
(1, 1, 0)
(6, 6, 0)
(6, 3, 0)
(3, 0, 0)
(3, -1, 0)
(3, -1, 0)
(4, -2, 0)
(6, -2, 0)
(5, -5, 0)
(4, -5, 0)
(3, -7, 0)
(4, -9, 0)
(4, -9, 0)
(4, -10, 0)
(6, -11, 0)
(6, -12, 0)
(7, -10, 0)
(6, -9, 0)
(5, -8, 0)
(5, -8, 0)
(2, -6, 0)
(3, -8, 0)
(2, -9, 0)
(1, -10, 0)
(0, -6, 0)
(0, -6, 0)
(0, -8, 0)
(0, -5, 0)
(0, -3, 0)
(-1, -4, 0)
(-2, -7, 0)
(0, -5, 0)
(0, 0, 7865344)
(0, 0, 7865344)
(0, 0, 7865344)
(0, 0, 7865344)
(0, 0, 7865344)
(0, -1, 0)
(-1, -4, 0)
(0, -2, 0)
(-1, -2, 0)
(0, -2, 0)

In [17]:
list_mouse

[(0, 3, 0),
 (7, 56, 0),
 (7, 33, 0),
 (9, 32, 0),
 (9, 36, 0),
 (10, 33, 0),
 (13, 37, 0),
 (10, 37, 0),
 (9, 37, 0),
 (6, 29, 0),
 (6, 29, 0),
 (6, 33, 0),
 (5, 26, 0),
 (4, 25, 0),
 (2, 21, 0),
 (2, 19, 0),
 (1, 16, 0),
 (2, 10, 0),
 (1, 13, 0),
 (1, 2, 0),
 (0, 3, 7865344),
 (1, 3, 0),
 (6, 45, 0),
 (0, 3, 0),
 (1, 3, 7865344),
 (0, 2, 0),
 (6, 37, 0),
 (1, 2, 7865344),
 (3, 17, 0),
 (4, 12, 0),
 (0, 2, 7865344),
 (0, 2, 0),
 (11, 25, 0),
 (3, 5, 0),
 (1, 1, 7865344),
 (1, 2, 0),
 (5, 8, 0),
 (4, 1, 0),
 (5, 4, 0),
 (2, 1, 0),
 (1, 0, 7865344),
 (1, 1, 0),
 (6, 6, 0),
 (6, 3, 0),
 (3, 0, 0),
 (3, -1, 0),
 (3, -1, 0),
 (4, -2, 0),
 (6, -2, 0),
 (5, -5, 0),
 (4, -5, 0),
 (3, -7, 0),
 (4, -9, 0),
 (4, -9, 0),
 (4, -10, 0),
 (6, -11, 0),
 (6, -12, 0),
 (7, -10, 0),
 (6, -9, 0),
 (5, -8, 0),
 (5, -8, 0),
 (2, -6, 0),
 (3, -8, 0),
 (2, -9, 0),
 (1, -10, 0),
 (0, -6, 0),
 (0, -6, 0),
 (0, -8, 0),
 (0, -5, 0),
 (0, -3, 0),
 (-1, -4, 0),
 (-2, -7, 0),
 (0, -5, 0),
 (0, 0, 7865344),
 (0, 0, 

In [18]:
'''
import time
start_time = time.time()

i = 0
while time.time() - start_time < 2:
    GetMessage(pmsg, None, 0, 0)
    TranslateMessage(pmsg)
    DispatchMessage(pmsg)
    i += 1
    time.sleep(0.001)
'''

'\nimport time\nstart_time = time.time()\n\ni = 0\nwhile time.time() - start_time < 2:\n    GetMessage(pmsg, None, 0, 0)\n    TranslateMessage(pmsg)\n    DispatchMessage(pmsg)\n    i += 1\n    time.sleep(0.001)\n'